In [ ]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import plotly.graph_objects as go
import json

# Worldwide wildfire choropleth animated map

In [ ]:
# Load csv file for wildfire data
wildfires = Path("resources/2- annual-area-burnt-by-wildfires.csv")
wildfire_df = pd.read_csv(wildfires)

wildfire_df.head()
wildfire_df.info()

In [ ]:
# Rename columns
wildfire_df = wildfire_df.rename(columns = {
    "Entity" : "Country", 
    "Code" : 'Alpha-3 code', 
    "Annual area burnt by wildfires" : "Acres Burned"
})

wildfire_df.head()

In [ ]:
# Drop null values
wildfire_df.dropna(inplace=True)
wildfire_df.info()

In [ ]:
# Separate DF by year
df_2012 = wildfire_df[wildfire_df["Year"] ==2012]
df_2013 = wildfire_df[wildfire_df["Year"] ==2013]
df_2014 = wildfire_df[wildfire_df["Year"] ==2014]
df_2015 = wildfire_df[wildfire_df["Year"] ==2015]
df_2016 = wildfire_df[wildfire_df["Year"] ==2016]
df_2017 = wildfire_df[wildfire_df["Year"] ==2017]
df_2018 = wildfire_df[wildfire_df["Year"] ==2018]
df_2019 = wildfire_df[wildfire_df["Year"] ==2019]
df_2020 = wildfire_df[wildfire_df["Year"] ==2020]
df_2021 = wildfire_df[wildfire_df["Year"] ==2021]
df_2022 = wildfire_df[wildfire_df["Year"] ==2022]
df_2023 = wildfire_df[wildfire_df["Year"] ==2023]

In [ ]:
# Import geographic data csv
coords = Path("resources/countries_codes_and_coordinates.csv")
coordinates_df = gpd.read_file(coords)

coordinates_df.head()

In [ ]:
coordinates_df.columns

In [ ]:
# Keep only necessary columns
coordinates_df = coordinates_df[['Country', 'Alpha-3 code', 'Numeric code',
       'Latitude (average)', 'Longitude (average)', 'geometry']]

coordinates_df.head()

In [ ]:
# Strip whitespace
coordinates_df = coordinates_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
coordinates_df.head()

In [ ]:
# Merge each year DF with geographic data
merged_2012 = pd.merge(df_2012, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2013 = pd.merge(df_2013, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2014 = pd.merge(df_2014, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2015 = pd.merge(df_2015, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2016 = pd.merge(df_2016, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2017 = pd.merge(df_2017, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2018 = pd.merge(df_2018, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2019 = pd.merge(df_2019, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2020 = pd.merge(df_2020, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2021 = pd.merge(df_2021, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2022 = pd.merge(df_2022, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2023 = pd.merge(df_2023, coordinates_df, how = "inner", on = ['Country', "Alpha-3 code"])
merged_2012.head()

In [ ]:
# Append each year DF to one DF to graph
new_fires = pd.concat([merged_2013, merged_2014, merged_2015, merged_2016, merged_2017, merged_2018, merged_2019, merged_2020, merged_2021, merged_2022, merged_2023], ignore_index = True)

In [ ]:
new_fires.to_csv('resources/new_fires.csv')

In [ ]:
# Create Choropleth Map with Alpha-3 Code, slider bar is animated by Year
fig = px.choropleth(
    new_fires,
    geojson=coordinates_df['Alpha-3 code'],
    locations='Alpha-3 code',
    color='Acres Burned',
    animation_frame='Year', 
    color_continuous_scale= px.colors.sequential.Reds,
    title='Area Burnt by Wildfires over Time', 
    height = 750, 
    width = 900
)
fig.update_geos(fitbounds="locations", visible=False) 
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 10000

# Export to HTML for project
fig.write_html("html/wildfiredata.html")
fig.show()

In [ ]:
# Filter the top 10 countries based on total acres burned
total_acres_burned = new_fires.groupby('Country')['Acres Burned'].sum().sort_values(ascending=False)

# Get the top 10 countries by total acres burned
top_10_countries = total_acres_burned.head(10).index

In [ ]:
# Filter the data to include only the top 10 countries
top_10_data = new_fires[new_fires['Country'].isin(top_10_countries)]


In [ ]:
# Aggregate the data by country and year (sum of acres burned)
pivot_data = top_10_data.groupby(['Country', 'Year'])['Acres Burned'].sum().reset_index()

# Pivot the data to create a matrix of 'Country' vs. 'Year' with 'Acres Burned' as values
heatmap_data = pivot_data.pivot(index='Country', columns='Year', values='Acres Burned')

# Sort the rows in descending order by total acres burned
heatmap_data = heatmap_data.loc[total_acres_burned.head(10).index]

In [ ]:
# Set up the plot size and style
plt.figure(figsize=(12, 8))

plt.gcf().set_facecolor('black')  # Set figure background to black
plt.gca().set_facecolor('black')

# Create the heatmap
heatmap = sns.heatmap(heatmap_data, annot=False, cmap='YlOrRd', cbar_kws={'label': 'Acres Burned (Millions)'}, color = 'white')

# Set plot labels
plt.title('Acres Burned by Top 10 Countries (2013-2023)', fontsize=16, color='white')
plt.xlabel('Year', fontsize=12, color='white')
plt.ylabel('Country', fontsize=12, color='white')


# Change color bar label to white
colorbar = plt.gca().collections[0].colorbar  # Get the colorbar
colorbar.set_label('Acres Burned (Millions)', fontsize=12, color='white')  # Set colorbar label to white
colorbar.ax.tick_params(labelcolor='white')  # Set color of colorbar ticks to white


plt.xticks(fontsize=10, color='white')
plt.yticks(fontsize=10, color='white')

# Show the plot
plt.savefig('html/fire_heatmap.png')
plt.show()
